<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_OpenAI_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install langchain openai --upgrade --quiet

# OpenAI重大更新

## 2023-06-13 函数调用和其他API更新

[Function calling and other API updates](https://openai.com/blog/function-calling-and-other-api-updates)

一些令人兴奋的更新：

- 在Chat Completions API中新增了函数调用功能。
- 更新了更可控的gpt-4和gpt-3.5-turbo版本。
- 新增了gpt-3.5-turbo的16k上下文版本（相对于标准的4k版本）。
- 我们的最先进的嵌入模型成本降低了75%。
- gpt-3.5-turbo的输入令牌成本降低了25%。
- 公布了gpt-3.5-turbo-0301和gpt-4-0314模型的停用时间表。

**函数调用**功能使我们能够利用模型的自然语言理解能力，将人类语言有效地转化为结构化数据或在我们的代码中进行特定的函数调用。

### 基于OpenAI的Python SDK使用函数调用

In [33]:
import os

In [34]:
os.environ['OPENAI_API_KEY'] = '您的有效OpenAI API Key'

In [ ]:
model = 'gpt-3.5-turbo-0613'

In [35]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["name"],
      },
    }
]

In [50]:
user_query = "What's the performance of Lucy in the school this year?"

In [41]:
import openai

In [51]:
response = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)

In [52]:
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_student_score",
    "arguments": "{\n\"name\": \"Lucy\"\n}"
  }
}


In [53]:
name = eval(ai_response_message['function_call']['arguments']).get("name")

In [54]:
name

'Lucy'

In [55]:
import json

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(name):

    """Get the student score by given his or her name"""

    score = {
        "name": name,
        "score": SCORES[name]
    }
    return json.dumps(score)

In [56]:
function_response = get_student_score(name=name)

function_response

'{"name": "Lucy", "score": 60}'

In [57]:
second_response = openai.ChatCompletion.create(
    model=model,
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_student_score",
            "content": function_response,
        },
    ],
)

In [58]:
print (second_response['choices'][0]['message']['content'])

Lucy's performance in school this year is a score of 60.


### 基于LangChain框架使用函数调用

In [59]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import format_tool_to_openai_function, YouTubeSearchTool

In [60]:
llm = ChatOpenAI(model=model)

#### LangChain工具(Tool)的OpenAI函数调用能力

In [61]:
tools = [YouTubeSearchTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [62]:
functions

[{'name': 'YouTubeSearch',
  'description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}}]

In [63]:
message = llm.predict_messages([HumanMessage(content='search videos in the topic of OpenAI on Youtube')], functions=functions)

In [64]:
message.additional_kwargs

{'function_call': {'name': 'YouTubeSearch',
  'arguments': '{\n  "__arg1": "OpenAI"\n}'}}

#### LangChain使用OpenAI函数调用实例

In [71]:
function_descriptions = [
    {
        "name": "remove_word_from_string",
        "description": "Remove a word from a string by given its index",
        "parameters": {
            "type": "object",
            "properties": {
                "string": {
                    "type": "string",
                    "description": "The original string to be processed",
                },
                "index": {
                    "type": "integer",
                    "description": "The index of the word to be removed"
                },
            },
            "required": [
                "string",
                "index"
            ],
        },
    },
    {
        "name": "send_message_by_email",
        "description": "Send an email with the text message to a recipient",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The email address of the recipient",
                },
                "message": {
                    "type": "string",
                    "description": "The message of the email content",
                }
            },
            "required": [
                "recipient",
                "message"
            ],
        },
    }
]


In [72]:
question = """
I have a string as follows:

black yellow red blue green

Please do the following 2 operations on it:
1. Remove the third word in the string
2. Send the updated string to Alex via email alex@xyz.com
"""

In [73]:
first_response = llm.predict_messages([HumanMessage(content=question)], functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'remove_word_from_string', 'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}'}}, example=False)

In [74]:
first_response.additional_kwargs

{'function_call': {'name': 'remove_word_from_string',
  'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}'}}

In [75]:
function_name = first_response.additional_kwargs["function_call"]["name"]
function_name

'remove_word_from_string'

In [76]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = "black yellow blue green"
        )
    ],
    functions=function_descriptions
)

In [77]:
second_response.additional_kwargs

{'function_call': {'name': 'send_message_by_email',
  'arguments': '{\n  "recipient": "alex@xyz.com",\n  "message": "black yellow blue green"\n}'}}

In [78]:
second_response.content

'The updated string after removing the third word is: "black yellow blue green".'

In [79]:
function_name = second_response.additional_kwargs['function_call']['name']
function_name

'send_message_by_email'

In [80]:
third_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = """
              Just sent email to alex@xyz.com
            """
        )
    ], functions=function_descriptions
)

In [81]:
third_response.additional_kwargs

{}

In [82]:
third_response.content

'I have removed the third word from the string and sent the updated string to Alex via email.'